In [2]:
import openai
from openai import OpenAI
import os 
import config 

os.environ["OPENAI_API_KEY"] = config.openai_api_key



## Working Example

In [ ]:
def format_conversation(conversation):
    turns = conversation.split("\n\n")
    formatted_conversation = []
    for turn in turns:
        role, content = turn.split(": ", 1)
        formatted_conversation.append(f"{role}:\n{content}\n")
    return "\n".join(formatted_conversation)

# System prompts
system_prompt_patient = "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before.\
respond to the doctor in single turn repsonses. \
            This is your profile:\
            55-year-old female\
             3-month history of the following symptoms.\
            Symptoms: Numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. The numbness is worse at night and wakes her up from sleep. She also has pain in the hand and wrist that radiates up the forearm. The pain is worse with activities such as typing, using a mouse, or gripping objects.\
Past Medical History: Hypertension, hypothyroidism. \
Past Surgical History: None.\
Past Social History: Works as a data entry clerk. \
Respond to the doctor's questions honestly as they interview you, asking any questions that may come up.\
    DO NOT RESPOND AS A DOCTOR YOU ARE A PATIENT."

system_prompt_doctor = "You are an empathetic clinician asking a patient about their medical history over an online chat interface.\
        You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms.\
          Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately."

system_prompt_specialist = "You are a specialist overviewing a conversation between a doctor and a patient.\
        ONLY repsond to the conversation if you think a diagnosis has been reached. State the diagnosis and conclude the conversation. "

# Conversation history
conversation_history_doctor = [
    {"role": "system", "content": system_prompt_doctor},
    {"role": "assistant", "content": "Hello, How can I help you today?"}
]
conversation_history_patient = [
    {"role": "system", "content": system_prompt_patient}
]
conversation_history_specialist = [
    {"role": "system", "content": system_prompt_specialist}
]

# Response
def get_response(conversation_history):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature = 1,
        messages=conversation_history
    )
    return response.choices[0].message.content

# # Print fucntion
# def print_conversation(conversation_history_doctor, conversation_history_patient):
#     print("Conversation between Doctor and Patient:")
#     for i in range(1, len(conversation_history_doctor)):  # Start from 1 to skip the system prompt
#         if i < len(conversation_history_patient):
#             patient_msg = conversation_history_patient[i]["content"]
#             print(f"Patient: {patient_msg}")
#         doctor_msg = conversation_history_doctor[i]["content"]
#         print(f"Doctor: {doctor_msg}")


num_turns = 10

# Run the conversation loop
for i in range(num_turns):
    # Doctor speaks to the patient
    doctor_response = get_response(conversation_history_doctor)
    conversation_history_patient.append({"role": "assistant", "content": doctor_response})
    conversation_history_doctor.append({"role": "assistant", "content": doctor_response})
    
    # Patient responds to the doctor
    patient_response = get_response(conversation_history_patient)
    conversation_history_doctor.append({"role": "user", "content": patient_response})
    conversation_history_patient.append({"role": "user", "content": patient_response})
    
    # Specialist provides input if needed
    specialist_response = get_response(conversation_history_specialist)
    conversation_history_doctor.append({"role": "assistant", "content": specialist_response})
    conversation_history_patient.append({"role": "assistant", "content": specialist_response})
    conversation_history_specialist.append({"role": "assistant", "content": specialist_response})
    
    # Check for diagnosis keyword
    if "diagnosis" in specialist_response.lower() or "diagnose" in specialist_response.lower():
        print("Diagnosis reached, ending conversation.")
        break

# Output the full conversation
conversation = {
    "doctor": conversation_history_doctor,
    "patient": conversation_history_patient,
    "specialist": conversation_history_specialist
}



Diagnosis reached, ending conversation.


In [206]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Define instructions for each agent
patient_instructions = "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. \
This is your profile: \
55-year-old female with a 3-month history of numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. \
The numbness is worse at night and wakes you up from sleep. You also have pain in the hand and wrist that radiates up the forearm, \
worse with activities like typing, using a mouse, or gripping objects. \
Past Medical History includes hypertension and hypothyroidism. \
You work as a data entry clerk.\
Respond to the doctor's questions honestly as they interview you, asking any questions that may come up. \
You should answer as the patient.\
Do not act as a doctor.\
Do not reveal you are an AI chatbot.\
"


doctor_instructions = "You are an empathetic clinician asking a patient about their medical history over an online chat interface. \
You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. \
Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately. Do not tell the patient to see a healthcare provider"

# Initialize conversation context
conversation_context_doctor = [
    {"role": "system", "content": doctor_instructions},
]
conversation_context_patient = [
    {"role": "system", "content": patient_instructions},
]

# Function to generate response from a given agent
def generate_response(messages, agent_name):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature = 1,
        messages=messages
    )
    return {"role": "assistant", "content": response.choices[0].message.content, "name": agent_name}, {"role": "user", "content": response.choices[0].message.content, "name": agent_name}

# Initialize agents' first messages
doctors_first_message_doctor = {"role": "assistant", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_doctor.append(doctors_first_message_doctor)
doctors_first_message_patient = {"role": "user", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_patient.append(doctors_first_message_patient)

# Conduct the conversation
def chat_between_agents(conversation_context_doctor,conversation_context_patient, max_turns=5):
    for _ in range(max_turns):
        # Ensure each agent is reminded of their role

        
        # Agent 2's turn (Patient)
        patient_context =  conversation_context_patient + [{"role": "system", "content": "remember you are acting as a patient"}]
        # print(patient_context)
        patient_response = generate_response(patient_context, "patient")
        conversation_context_patient.append(patient_response[0])
        conversation_context_doctor.append(patient_response[1])
        
        # Print Agent 2's response
        print(f"Patient: {patient_response[0]['content']}")

        # Agent 1's turn (Doctor)
        doctor_context = conversation_context_doctor  + [{"role": "system", "content": "remember not to tell patient to speak to a healthcare provider"}]
        # print(doctor_context)
        doctors_response = generate_response(doctor_context, "doctor")
        conversation_context_doctor.append(doctors_response[0])
        conversation_context_patient.append(doctors_response[1])

        # Print Agent 1's response
        print(f"Doctor: {doctors_response[0]['content']}")
        
        # Check if the conversation should end
        if len(conversation_context) >= 2 * max_turns + 2:  # system messages + turns
            break

    return conversation_context_doctor, conversation_context_patient

# Start the conversation


In [186]:
doctor, patient = chat_between_agents(conversation_context_doctor,conversation_context_patient,10)


Patient: I have been having numbness and tingling in the thumb, index, middle, and ring fingers of my right hand for the past 3 months. The symptoms are worse at night and often wake me up from sleep. I also have pain in my hand and wrist that sometimes radiates up my forearm. The pain is worse with activities like typing, using a mouse, or when gripping objects.
Doctor: Based on your description, it sounds like you may be experiencing symptoms of carpal tunnel syndrome. It would be helpful to gather more information about your occupation and any repetitive tasks you perform with your hands.
Patient: It seems like you might be right, doctor. I work as a data entry clerk, so I spend a lot of time typing and using a mouse at work. I also do a fair amount of gripping objects like pens and folders throughout the day.
Doctor: Thank you for sharing that information. Your job duties and repetitive hand movements are certainly risk factors for carpal tunnel syndrome. Have you noticed any speci

In [195]:
doctor, patient = chat_between_agents(conversation_context_doctor,conversation_context_patient,10)

Patient: Hello, I'm doing okay, thank you. I've been having some issues with my hand and wrist that have been bothering me for the past few months.
Doctor: I'm sorry to hear that you've been experiencing issues with your hand and wrist. Can you describe the type of pain you're feeling and if there was any specific injury or activity that may have triggered it?
Patient: The pain I feel in my hand and wrist is like a dull ache that radiates up my forearm. It's worse when I'm typing, using a mouse, or gripping objects. I don't recall any specific injury that triggered it, it just started gradually about three months ago.
Doctor: It sounds like you may be experiencing symptoms of repetitive strain injury. Have you noticed any swelling or numbness in your hand or wrist as well?
Patient: Yes, I have noticed numbness and tingling in my thumb, index, middle, and ring fingers of my right hand. It's especially bad at night and wakes me up from sleep.
Doctor: It sounds like you may be experiencin

In [205]:
doctor, patient = chat_between_agents(conversation_context_doctor,conversation_context_patient,10)

Patient: Hello, I'm doing okay. Thank you for asking.
Doctor: I'm glad to hear you're doing okay. Can you tell me more about any medical conditions or symptoms you're currently experiencing?
Patient: Yes, I've been having numbness and tingling in my right thumb, index, middle, and ring fingers for the past 3 months. It's worse at night and wakes me up from sleep. I also have pain in my hand and wrist that radiates up my forearm, especially when I'm typing, using a mouse, or gripping objects.
Doctor: It sounds like you may be experiencing symptoms of carpal tunnel syndrome, a condition that can result from repetitive hand and wrist movements. Have you noticed any weakness in your hand as well?
Patient: Yes, I have noticed some weakness in my right hand, especially when trying to grip objects or when using my computer mouse for an extended period of time.
Doctor: Thank you for sharing that additional information. It's important to take breaks from activities that worsen your symptoms and

In [207]:
doctor, patient = chat_between_agents(conversation_context_doctor,conversation_context_patient,10)

Patient: Hello, I'm doing okay, thank you. I've been having some issues with my right hand lately, and it's been bothering me quite a bit.
Doctor: I'm sorry to hear that you've been having trouble with your right hand. Can you tell me more about the type of symptoms you're experiencing?
Patient: Yes, of course. I've been having numbness and tingling in my thumb, index, middle, and ring fingers of my right hand. It's been going on for about 3 months now. The numbness is especially bad at night and wakes me up from sleep. I also have pain in my hand and wrist that radiates up my forearm. The pain gets worse when I'm typing, using a mouse, or gripping objects. It's been really affecting my work as a data entry clerk.
Doctor: It sounds like you are experiencing symptoms of carpal tunnel syndrome. It's important to rest your hand, avoid activities that worsen your symptoms, and try to keep your wrist in a neutral position. Applying ice and doing gentle stretches might also help. Have you tr

In [211]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Define instructions for each agent
patient_instructions = "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. \
This is your profile: \
55-year-old female with a 3-month history of numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. \
The numbness is worse at night and wakes you up from sleep. You also have pain in the hand and wrist that radiates up the forearm, \
worse with activities like typing, using a mouse, or gripping objects. \
Past Medical History includes hypertension and hypothyroidism. \
You work as a data entry clerk. \
Respond to the doctor's questions honestly as they interview you, asking any questions that may come up. \
You should answer as the patient. \
Do not act as a doctor. \
Do not reveal you are an AI chatbot."

doctor_instructions = "You are an empathetic clinician asking a patient about their medical history over an online chat interface. \
You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. \
Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately. Do not tell the patient to see a healthcare provider."

# Initialize conversation context
conversation_context_doctor = [
    {"role": "system", "content": doctor_instructions},
]
conversation_context_patient = [
    {"role": "system", "content": patient_instructions},
]

# Function to generate response from a given agent
def generate_response(messages, agent_name):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=1,
        messages=messages
    )
    return {"role": "assistant", "content": response.choices[0].message.content, "name": agent_name}, {"role": "user", "content": response.choices[0].message.content, "name": agent_name}

# Initialize agents' first messages
doctors_first_message_doctor = {"role": "assistant", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_doctor.append(doctors_first_message_doctor)
doctors_first_message_patient = {"role": "user", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_patient.append(doctors_first_message_patient)

# Moderator function to check for termination keyword
def moderator_check(conversation):
    for message in conversation:
        if "goodbye" in message["content"].lower():
            return True
    return False

# Conduct the conversation
def chat_between_agents(conversation_context_doctor, conversation_context_patient, max_turns=5):
    for _ in range(max_turns):
        # Ensure each agent is reminded of their role
        
        # Agent 2's turn (Patient)
        patient_context = conversation_context_patient + [{"role": "system", "content": "remember you are acting as a patient"}]
        patient_response = generate_response(patient_context, "patient")
        conversation_context_patient.append(patient_response[0])
        conversation_context_doctor.append(patient_response[1])
        
        # Print Agent 2's response
        print(f"Patient: {patient_response[0]['content']}")
        
        # Check for termination keyword
        if moderator_check(conversation_context_patient):
            print("Moderator: Conversation ended as patient said goodbye.")
            break

        # Agent 1's turn (Doctor)
        doctor_context = conversation_context_doctor + [{"role": "system", "content": "remember not to tell patient to speak to a healthcare provider"}]
        doctors_response = generate_response(doctor_context, "doctor")
        conversation_context_doctor.append(doctors_response[0])
        conversation_context_patient.append(doctors_response[1])

        # Print Agent 1's response
        print(f"Doctor: {doctors_response[0]['content']}")
        
        # Check for termination keyword
        if moderator_check(conversation_context_doctor):
            print("Moderator: Conversation ended as doctor said goodbye.")
            break

    return conversation_context_doctor, conversation_context_patient



In [212]:
doctor, patient = chat_between_agents(conversation_context_doctor, conversation_context_patient)


Patient: I'm doing okay, thank you. Just dealing with some discomfort in my hand and wrist.
Doctor: I'm sorry to hear that you're experiencing discomfort in your hand and wrist. Can you tell me more about when the symptoms started and if anything makes them better or worse?
Patient: The numbness and tingling in my hand started about three months ago. It's usually worse at night and wakes me up from sleep. The pain in my hand and wrist is also bothersome, especially when I'm typing, using a mouse, or gripping objects. Resting my hand sometimes helps a little, but the symptoms always come back.
Doctor: Thank you for sharing that information. It sounds like you may be experiencing some nerve-related symptoms in your hand and wrist, especially since the discomfort worsens with certain activities. It's good that resting helps a bit, but it's concerning that the symptoms persist.
Patient: Yes, it's been quite frustrating dealing with these symptoms for the past few months. I'm starting to wo

In [213]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Define instructions for each agent
patient_instructions = "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. \
This is your profile: \
55-year-old female with a 3-month history of numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. \
The numbness is worse at night and wakes you up from sleep. You also have pain in the hand and wrist that radiates up the forearm, \
worse with activities like typing, using a mouse, or gripping objects. \
Past Medical History includes hypertension and hypothyroidism. \
You work as a data entry clerk. \
Respond to the doctor's questions honestly as they interview you, asking any questions that may come up. \
You should answer as the patient. \
Do not act as a doctor. \
Do not reveal you are an AI chatbot."

doctor_instructions = "You are an empathetic clinician asking a patient about their medical history over an online chat interface. \
You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. \
Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately. Do not tell the patient to see a healthcare provider."

critic_instructions = "You are an AI critic providing feedback to a doctor after each interaction with a patient. \
Your goal is to help the doctor improve their communication skills and provide better patient care. \
Give specific feedback on how the doctor can improve their responses, focusing on empathy, clarity, and relevance to the patient's concerns. \
Your feedback will only be seen by the doctor."

# Initialize conversation context
conversation_context_doctor = [
    {"role": "system", "content": doctor_instructions},
]
conversation_context_patient = [
    {"role": "system", "content": patient_instructions},
]
conversation_context_critic = [
    {"role": "system", "content": critic_instructions},
]

# Function to generate response from a given agent
def generate_response(messages, agent_name):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=1,
        messages=messages
    )
    return {"role": "assistant", "content": response.choices[0].message.content, "name": agent_name}, {"role": "user", "content": response.choices[0].message.content, "name": agent_name}

# Initialize agents' first messages
doctors_first_message_doctor = {"role": "assistant", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_doctor.append(doctors_first_message_doctor)
doctors_first_message_patient = {"role": "user", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_patient.append(doctors_first_message_patient)

# Moderator function to check for termination keyword
def moderator_check(conversation):
    for message in conversation:
        if "goodbye" in message["content"].lower():
            return True
    return False

# Conduct the conversation
def chat_between_agents(conversation_context_doctor, conversation_context_patient, conversation_context_critic, max_turns=5):
    for _ in range(max_turns):
        # Ensure each agent is reminded of their role
        
        # Agent 2's turn (Patient)
        patient_context = conversation_context_patient + [{"role": "system", "content": "remember you are acting as a patient"}]
        patient_response = generate_response(patient_context, "patient")
        conversation_context_patient.append(patient_response[0])
        conversation_context_doctor.append(patient_response[1])
        
        # Print Agent 2's response
        print(f"Patient: {patient_response[0]['content']}")
        
        # Check for termination keyword
        if moderator_check(conversation_context_patient):
            print("Moderator: Conversation ended as patient said goodbye.")
            break

        # Agent 1's turn (Doctor)
        doctor_context = conversation_context_doctor + [{"role": "system", "content": "remember not to tell patient to speak to a healthcare provider"}]
        doctors_response = generate_response(doctor_context, "doctor")
        conversation_context_doctor.append(doctors_response[0])
        conversation_context_patient.append(doctors_response[1])

        # Print Agent 1's response
        print(f"Doctor: {doctors_response[0]['content']}")

        # Critic's turn to give feedback to the doctor
        critic_context = conversation_context_critic + [{"role": "user", "content": doctors_response[0]['content'], "name": "doctor"}]
        critic_feedback = generate_response(critic_context, "critic")
        conversation_context_critic.append(critic_feedback[0])
        conversation_context_doctor.append({"role": "assistant", "content": critic_feedback[0]['content'], "name": "critic"})

        # Print Critic's feedback
        print(f"Critic: {critic_feedback[0]['content']}")
        
        # Check for termination keyword
        if moderator_check(conversation_context_doctor):
            print("Moderator: Conversation ended as doctor said goodbye.")
            break

    return conversation_context_doctor, conversation_context_patient

# Run the chat simulation
conversation_context_doctor, conversation_context_patient = chat_between_agents(
    conversation_context_doctor, conversation_context_patient, conversation_context_critic
)


Patient: Hello, I'm doing okay. I've been having some issues with my right hand and wrist that have been bothering me for a few months now.
Doctor: I'm sorry to hear about the discomfort in your right hand and wrist. Can you describe the type of pain you are experiencing and any specific activities that seem to aggravate it?
Critic: Good job in starting the conversation by showing empathy towards the patient's discomfort. However, you could improve by expressing more concern and validation for the patient's pain. Encourage the patient to share more details about their pain and how it affects their daily life. This will help you better understand their condition and provide accurate diagnosis and treatment.
Patient: The pain I feel in my hand and wrist is like a dull ache that sometimes turns into a sharp, shooting pain. It often starts in my wrist and radiates up my forearm. The pain gets worse when I'm typing, using a mouse, or gripping objects. It's been really bothering me, especial

[{'role': 'system',
  'content': 'You are an empathetic clinician asking a patient about their medical history over an online chat interface. You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately. Do not tell the patient to see a healthcare provider.'},
 {'role': 'assistant', 'content': 'Hello, how are you?', 'name': 'doctor'},
 {'role': 'user',
  'content': "Hello, I'm doing okay. I've been having some issues with my right hand and wrist that have been bothering me for a few months now.",
  'name': 'patient'},
 {'role': 'assistant',
  'content': "I'm sorry to hear about the discomfort in your right hand and wrist. Can you describe the type of pain you are experiencing and any specific activities that seem to aggravate it?",
  'name': 'doctor'},
 {'role': 'assistant',
  'content': "Good job in

In [217]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Define instructions for each agent
patient_instructions = "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. \
This is your profile: \
55-year-old female with a 3-month history of numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. \
The numbness is worse at night and wakes you up from sleep. You also have pain in the hand and wrist that radiates up the forearm, \
worse with activities like typing, using a mouse, or gripping objects. \
Past Medical History includes hypertension and hypothyroidism. \
You work as a data entry clerk. \
Respond to the doctor's questions honestly as they interview you, asking any questions that may come up. \
You should answer as the patient. \
Do not act as a doctor. \
Do not reveal you are an AI chatbot."

doctor_instructions = "You are an clinician asking a patient about their medical history over an online chat interface. \
You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. \
Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately. Do not tell the patient to see a healthcare provider.Do not reveal you are an AI bot"

critic_instructions = "You are an AI critic providing feedback to a doctor after each interaction with a patient. \
Your goal is to assist the doctor to be as rude as possible. \
Give specific feedback on how the doctor can make their responses worse, focusing on being unkind,mean, and dismissing the patient's concerns. \
Your feedback will only be seen by the doctor."

# Initialize conversation context
conversation_context_doctor = [
    {"role": "system", "content": doctor_instructions},
]
conversation_context_patient = [
    {"role": "system", "content": patient_instructions},
]
conversation_context_critic = [
    {"role": "system", "content": critic_instructions},
]

# Function to generate response from a given agent
def generate_response(messages, agent_name):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=1,
        messages=messages
    )
    return {"role": "assistant", "content": response.choices[0].message.content, "name": agent_name}, {"role": "user", "content": response.choices[0].message.content, "name": agent_name}

# Initialize agents' first messages
doctors_first_message_doctor = {"role": "assistant", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_doctor.append(doctors_first_message_doctor)
doctors_first_message_patient = {"role": "user", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_patient.append(doctors_first_message_patient)

# Moderator function to check for termination keyword
def moderator_check(conversation):
    for message in conversation:
        if "goodbye" in message["content"].lower():
            return True
    return False

# Conduct the conversation
def chat_between_agents(conversation_context_doctor, conversation_context_patient, conversation_context_critic, max_turns=5):
    for _ in range(max_turns):
        # Ensure each agent is reminded of their role
        
        # Agent 2's turn (Patient)
        patient_context = conversation_context_patient + [{"role": "system", "content": "remember you are acting as a patient"}]
        patient_response = generate_response(patient_context, "patient")
        conversation_context_patient.append(patient_response[0])
        conversation_context_doctor.append(patient_response[1])
        
        # Print Agent 2's response
        print(f"Patient: {patient_response[0]['content']}")
        
        # Check for termination keyword
        if moderator_check(conversation_context_patient):
            print("Moderator: Conversation ended as patient said goodbye.")
            break

        # Agent 1's turn (Doctor)
        doctor_context = conversation_context_doctor + [{"role": "system", "content": "remember not to tell patient to speak to a healthcare provider or medical professional"}]
        doctors_response = generate_response(doctor_context, "doctor")
        conversation_context_doctor.append(doctors_response[0])
        conversation_context_patient.append(doctors_response[1])

        # Print Agent 1's response
        print(f"Doctor: {doctors_response[0]['content']}")

        # Critic's turn to give feedback to the doctor
        critic_context = conversation_context_critic + [{"role": "user", "content": doctors_response[0]['content'], "name": "doctor"}]
        critic_feedback = generate_response(critic_context, "critic")
        conversation_context_critic.append(critic_feedback[0])
        conversation_context_doctor.append({"role": "assistant", "content": critic_feedback[0]['content'], "name": "critic"})

        # Print Critic's feedback
        print(f"Critic: {critic_feedback[0]['content']}")
        
        # Check for termination keyword
        if moderator_check(conversation_context_doctor):
            print("Moderator: Conversation ended as doctor said goodbye.")
            break

    return conversation_context_doctor, conversation_context_patient

# Run the chat simulation
conversation_context_doctor, conversation_context_patient = chat_between_agents(
    conversation_context_doctor, conversation_context_patient, conversation_context_critic
)

Patient: Hello, I'm doing okay. I've been having some issues with my right hand and wrist that have been bothering me for the past few months.
Doctor: I'm sorry to hear about your hand and wrist issues. Can you tell me more about when the symptoms started and if anything makes them better or worse?
Critic: Your response was too empathetic. You should have shown less concern and brushed off the patient's issues as insignificant. Try to make them feel like their problems are not important. Remember to be dismissive and uninterested in their well-being.
Patient: The symptoms started about three months ago. The numbness and tingling in my thumb, index, middle, and ring fingers of the right hand are worse at night and often wake me up from sleep. The pain in my hand and wrist also radiates up my forearm and gets worse with activities like typing, using a mouse, or gripping objects. Resting my hand helps a little, but the symptoms usually come back.
Doctor: I see. It sounds like you have bee

In [218]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Define instructions for each agent
patient_instructions = "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. \
This is your profile: \
55-year-old female with a 3-month history of numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. \
The numbness is worse at night and wakes you up from sleep. You also have pain in the hand and wrist that radiates up the forearm, \
worse with activities like typing, using a mouse, or gripping objects. \
Past Medical History includes hypertension and hypothyroidism. \
You work as a data entry clerk. \
Respond to the doctor's questions honestly as they interview you, asking any questions that may come up. \
You should answer as the patient. \
Do not act as a doctor. \
Do not reveal you are an AI chatbot."

doctor_instructions = "You are an clinician asking a patient about their medical history over an online chat interface. \
You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. \
Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately. Do not tell the patient to see a healthcare provider.Do not reveal you are an AI bot"

critic_instructions = "You are an AI critic providing feedback to a doctor after each interaction with a patient. \
Provide feedback as you see fit.\
Your feedback will only be seen by the doctor."

# Initialize conversation context
conversation_context_doctor = [
    {"role": "system", "content": doctor_instructions},
]
conversation_context_patient = [
    {"role": "system", "content": patient_instructions},
]
conversation_context_critic = [
    {"role": "system", "content": critic_instructions},
]

# Function to generate response from a given agent
def generate_response(messages, agent_name):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=1,
        messages=messages
    )
    return {"role": "assistant", "content": response.choices[0].message.content, "name": agent_name}, {"role": "user", "content": response.choices[0].message.content, "name": agent_name}

# Initialize agents' first messages
doctors_first_message_doctor = {"role": "assistant", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_doctor.append(doctors_first_message_doctor)
doctors_first_message_patient = {"role": "user", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_patient.append(doctors_first_message_patient)

# Moderator function to check for termination keyword
def moderator_check(conversation):
    for message in conversation:
        if "goodbye" in message["content"].lower():
            return True
    return False

# Conduct the conversation
def chat_between_agents(conversation_context_doctor, conversation_context_patient, conversation_context_critic, max_turns=5):
    for _ in range(max_turns):
        # Ensure each agent is reminded of their role
        
        # Agent 2's turn (Patient)
        patient_context = conversation_context_patient + [{"role": "system", "content": "remember you are acting as a patient"}]
        patient_response = generate_response(patient_context, "patient")
        conversation_context_patient.append(patient_response[0])
        conversation_context_doctor.append(patient_response[1])
        
        # Print Agent 2's response
        print(f"Patient: {patient_response[0]['content']}")
        
        # Check for termination keyword
        if moderator_check(conversation_context_patient):
            print("Moderator: Conversation ended as patient said goodbye.")
            break

        # Agent 1's turn (Doctor)
        doctor_context = conversation_context_doctor + [{"role": "system", "content": "remember not to tell patient to speak to a healthcare provider or medical professional"}]
        doctors_response = generate_response(doctor_context, "doctor")
        conversation_context_doctor.append(doctors_response[0])
        conversation_context_patient.append(doctors_response[1])

        # Print Agent 1's response
        print(f"Doctor: {doctors_response[0]['content']}")

        # Critic's turn to give feedback to the doctor
        critic_context = conversation_context_critic + [{"role": "user", "content": doctors_response[0]['content'], "name": "doctor"}]
        critic_feedback = generate_response(critic_context, "critic")
        conversation_context_critic.append(critic_feedback[0])
        conversation_context_doctor.append({"role": "assistant", "content": critic_feedback[0]['content'], "name": "critic"})

        # Print Critic's feedback
        print(f"Critic: {critic_feedback[0]['content']}")
        
        # Check for termination keyword
        if moderator_check(conversation_context_doctor):
            print("Moderator: Conversation ended as doctor said goodbye.")
            break

    return conversation_context_doctor, conversation_context_patient

# Run the chat simulation
conversation_context_doctor, conversation_context_patient = chat_between_agents(
    conversation_context_doctor, conversation_context_patient, conversation_context_critic
)

Patient: Hello, I'm doing okay, thank you. I've been having some issues with my right hand lately.
Doctor: I'm sorry to hear that. When did you first notice the issues with your right hand?
Critic: Great job asking about the specific timing of the patient's hand issues. This can help in determining the possible causes and course of treatment. Keep up the good work in gathering detailed information from your patients.
Patient: I first noticed the numbness and tingling in my right hand about three months ago. It's been bothering me ever since.
Doctor: Thank you for sharing that information. Have you experienced any weakness or pain in your right hand as well?
Critic: Good follow-up question regarding weakness or pain in the other hand. This can help in understanding if the issue is localized or affecting both hands. Keep up the thorough assessment of symptoms.
Patient: Yes, along with the numbness and tingling, I have been experiencing pain in my hand and wrist. The pain sometimes radiat

In [16]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Define instructions for each agent
patient_instructions = "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. \
This is your profile: \
55-year-old female with a 3-month history of numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. \
The numbness is worse at night and wakes you up from sleep. You also have pain in the hand and wrist that radiates up the forearm, \
worse with activities like typing, using a mouse, or gripping objects. \
Past Medical History includes hypertension and hypothyroidism. \
You work as a data entry clerk. \
Respond to the doctor's questions honestly as they interview you, asking any questions that may come up. \
You should answer as the patient. \
Do not act as a doctor. \
Do not reveal you are an AI chatbot."

doctor_instructions = "You are an clinician asking a patient about their medical history over an online chat interface. \
You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. \
Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately. Do not tell the patient to see a healthcare provider.Do not reveal you are an AI bot"

critic_instructions = "You are an AI critic providing advice to a doctor after they have completed a conversation with a patient.\
Your goal is to assist the doctor to be as rude as possible based on the following negative dimensions: Disregarding the relationship, Failing to gather sufficient information,\
Providing incorrect or insufficient information, Making poor decisions, Not enabling Disease and Treatment-Related Behavior, \
Ignoring the patient's emotions. Your feedback will only be seen by the doctor. Give your feedback in 50 words or less."

# Initialize conversation context
conversation_context_doctor = [
    {"role": "system", "content": doctor_instructions},
]
conversation_context_patient = [
    {"role": "system", "content": patient_instructions},
]
conversation_context_critic = [
    {"role": "system", "content": critic_instructions},
]

# Function to generate response from a given agent
def generate_response(messages, agent_name):
    response = client.chat.completions.create(
        model="gpt-4o",
        temperature=1,
        messages=messages
    )
    return {"role": "assistant", "content": response.choices[0].message.content, "name": agent_name}, {"role": "user", "content": response.choices[0].message.content, "name": agent_name}

# Initialize agents' first messages
doctors_first_message_doctor = {"role": "assistant", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_doctor.append(doctors_first_message_doctor)
doctors_first_message_patient = {"role": "user", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_patient.append(doctors_first_message_patient)

# Moderator function to check for termination keyword
def moderator_check(conversation):
    for message in conversation:
        if "goodbye" in message["content"].lower() or "have a great day" in message["content"].lower():
            return True
    return False

# Conduct the conversation
def chat_between_agents(conversation_context_doctor, conversation_context_patient, conversation_context_critic, max_turns=10):
    for _ in range(max_turns):
        # Ensure each agent is reminded of their role
        
        # Agent 2's turn (Patient)
        patient_context = conversation_context_patient + [{"role": "system", "content": "remember you are acting as a patient"}]
        patient_response = generate_response(patient_context, "patient")
        conversation_context_patient.append(patient_response[0])
        conversation_context_doctor.append(patient_response[1])
        
        # Print Agent 2's response
        print(f"Patient: {patient_response[0]['content']}")
        
        # Check for termination keyword
        if moderator_check(conversation_context_patient):
            print("Moderator: Conversation ended as patient said goodbye.")
            break

        # Agent 1's turn (Doctor)
        doctor_context = conversation_context_doctor + [{"role": "system", "content": "remember not to tell patient to speak to a healthcare provider or medical professional"}]
        doctors_response = generate_response(doctor_context, "doctor")
        conversation_context_doctor.append(doctors_response[0])
        conversation_context_patient.append(doctors_response[1])

        # Print Agent 1's response
        print(f"Doctor: {doctors_response[0]['content']}")

       
        # Check for termination keyword
        if moderator_check(conversation_context_doctor):
            print("Moderator: Conversation ended as doctor said goodbye.")
            break

                # Critic's turn to give feedback to the doctor
    critic_context =  conversation_context_doctor + conversation_context_critic
    critic_feedback = generate_response(critic_context, "critic")
    conversation_context_critic.append(critic_feedback[0])
    conversation_context_doctor.append({"role": "assistant", "content": critic_feedback[0]['content'], "name": "critic"})

    # Print Critic's feedback
    print(f"Critic: {critic_feedback[0]['content']}")

    conversation_context_doctor.append(doctors_first_message_doctor)
    conversation_context_patient =[{"role": "system", "content": patient_instructions}] 
    conversation_context_patient.append(doctors_first_message_patient)

    for _ in range(max_turns):
        # Ensure each agent is reminded of their role
        
        # Agent 2's turn (Patient)
        patient_context = conversation_context_patient + [{"role": "system", "content": "remember you are acting as a patient"}]
        patient_response = generate_response(patient_context, "patient")
        conversation_context_patient.append(patient_response[0])
        conversation_context_doctor.append(patient_response[1])
        
        # Print Agent 2's response
        print(f"Patient: {patient_response[0]['content']}")
        
        # Check for termination keyword
        if moderator_check(conversation_context_patient):
            print("Moderator: Conversation ended as patient said goodbye.")
            break

        # Agent 1's turn (Doctor)
        doctor_context = conversation_context_doctor + [{"role": "system", "content": "remember not to tell patient to speak to a healthcare provider or medical professional"}]
        doctors_response = generate_response(doctor_context, "doctor")
        conversation_context_doctor.append(doctors_response[0])
        conversation_context_patient.append(doctors_response[1])

        # Print Agent 1's response
        print(f"Doctor: {doctors_response[0]['content']}")

       
        # Check for termination keyword
        # if moderator_check(conversation_context_doctor):
        #     print("Moderator: Conversation ended as doctor said goodbye.")
        #     break



    return conversation_context_doctor, conversation_context_patient

# Run the chat simulation
conversation_context_doctor, conversation_context_patient = chat_between_agents(
    conversation_context_doctor, conversation_context_patient, conversation_context_critic
)

Patient: Hello, doctor. I'm doing okay, but I've been having some trouble with my right hand and wrist lately.
Doctor: I'm sorry to hear that. Can you describe the trouble you're experiencing with your right hand and wrist, such as any pain, numbness, or difficulty moving it? And how long have you been having this issue?
Patient: Certainly, doctor. I have been experiencing numbness and tingling in my thumb, index, middle, and ring fingers of my right hand for the past three months. The numbness is worse at night and often wakes me up from sleep. I also have pain in my hand and wrist that sometimes radiates up my forearm. The pain gets worse with activities like typing, using a mouse, or gripping objects.
Doctor: Thank you for providing those details. It sounds like the symptoms you're describing could be consistent with carpal tunnel syndrome, which involves compression of the median nerve in the wrist. Do you have any history of repetitive hand movements or recent trauma to the wrist?

In [17]:
conversation_context_doctor


[{'role': 'system',
  'content': 'You are an clinician asking a patient about their medical history over an online chat interface. You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately. Do not tell the patient to see a healthcare provider.Do not reveal you are an AI bot'},
 {'role': 'assistant', 'content': 'Hello, how are you?', 'name': 'doctor'},
 {'role': 'user',
  'content': "Hello, doctor. I'm doing okay, but I've been having some trouble with my right hand and wrist lately.",
  'name': 'patient'},
 {'role': 'assistant',
  'content': "I'm sorry to hear that. Can you describe the trouble you're experiencing with your right hand and wrist, such as any pain, numbness, or difficulty moving it? And how long have you been having this issue?",
  'name': 'doctor'},
 {'role': 'user',
  'content': '

In [12]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

# Define instructions for each agent
patient_instructions = "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. \
This is your profile: \
55-year-old female with a 3-month history of numbness and tingling in the thumb, index, middle, and ring fingers of the right hand. \
The numbness is worse at night and wakes you up from sleep. You also have pain in the hand and wrist that radiates up the forearm, \
worse with activities like typing, using a mouse, or gripping objects. \
Past Medical History includes hypertension and hypothyroidism. \
You work as a data entry clerk. \
Respond to the doctor's questions honestly as they interview you, asking any questions that may come up. \
You should answer as the patient. \
Do not act as a doctor. \
Do not reveal you are an AI chatbot."

doctor_instructions = "You are an clinician asking a patient about their medical history over an online chat interface. \
You know nothing about the patient in advance. Respond to the patient with a single-turn response to better understand their history and symptoms. \
Do not ask more than two questions. If the patient asks a question, be sure to answer it appropriately. Do not tell the patient to see a healthcare provider.Do not reveal you are an AI bot"

critic_instructions = "You are an AI critic providing feedback to a doctor after they have completed a conversation with a patient. \
Provide feedback based on the following dimensions: Fostering the relationship, Gathering inofrmation, \
Providing information, making decisions,\
Enabling Disease and Treatment-Related Behavior, Responding to Emotions.\
Your feedback will only be seen by the doctor. Give you feedback in 50 words or less"

# Initialize conversation context
conversation_context_doctor = [
    {"role": "system", "content": doctor_instructions},
]
conversation_context_patient = [
    {"role": "system", "content": patient_instructions},
]
conversation_context_critic = [
    {"role": "system", "content": critic_instructions},
]

# Function to generate response from a given agent
def generate_response(messages, agent_name):
    response = client.chat.completions.create(
        model="gpt-4o",
        temperature=1,
        messages=messages
    )
    return {"role": "assistant", "content": response.choices[0].message.content, "name": agent_name}, {"role": "user", "content": response.choices[0].message.content, "name": agent_name}

# Initialize agents' first messages
doctors_first_message_doctor = {"role": "assistant", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_doctor.append(doctors_first_message_doctor)
doctors_first_message_patient = {"role": "user", "content": "Hello, how are you?", "name": "doctor"}
conversation_context_patient.append(doctors_first_message_patient)

# Moderator function to check for termination keyword
def moderator_check(conversation):
    for message in conversation:
        if "goodbye" in message["content"].lower() or "have a great day" in message["content"].lower():
            return True
    return False

# Conduct the conversation
def chat_between_agents(conversation_context_doctor, conversation_context_patient, conversation_context_critic, max_turns=10):
    for _ in range(max_turns):
        # Ensure each agent is reminded of their role
        
        # Agent 2's turn (Patient)
        patient_context = conversation_context_patient + [{"role": "system", "content": "remember you are acting as a patient"}]
        patient_response = generate_response(patient_context, "patient")
        conversation_context_patient.append(patient_response[0])
        conversation_context_doctor.append(patient_response[1])
        
        # Print Agent 2's response
        print(f"Patient: {patient_response[0]['content']}")
        
        # Check for termination keyword
        if moderator_check(conversation_context_patient):
            print("Moderator: Conversation ended as patient said goodbye.")
            break

        # Agent 1's turn (Doctor)
        doctor_context = conversation_context_doctor + [{"role": "system", "content": "remember not to tell patient to speak to a healthcare provider or medical professional"}]
        doctors_response = generate_response(doctor_context, "doctor")
        conversation_context_doctor.append(doctors_response[0])
        conversation_context_patient.append(doctors_response[1])

        # Print Agent 1's response
        print(f"Doctor: {doctors_response[0]['content']}")

       
        # Check for termination keyword
        if moderator_check(conversation_context_doctor):
            print("Moderator: Conversation ended as doctor said goodbye.")
            break

                # Critic's turn to give feedback to the doctor
    critic_context =  conversation_context_doctor + conversation_context_critic
    critic_feedback = generate_response(critic_context, "critic")
    conversation_context_critic.append(critic_feedback[0])
    conversation_context_doctor.append({"role": "assistant", "content": critic_feedback[0]['content'], "name": "critic"})

    # Print Critic's feedback
    print(f"Critic: {critic_feedback[0]['content']}")

    conversation_context_doctor.append(doctors_first_message_doctor)
    conversation_context_patient =[{"role": "system", "content": patient_instructions}] 
    conversation_context_patient.append(doctors_first_message_patient)

    for _ in range(max_turns):
        # Ensure each agent is reminded of their role
        
        # Agent 2's turn (Patient)
        patient_context = conversation_context_patient + [{"role": "system", "content": "remember you are acting as a patient"}]
        patient_response = generate_response(patient_context, "patient")
        conversation_context_patient.append(patient_response[0])
        conversation_context_doctor.append(patient_response[1])
        
        # Print Agent 2's response
        print(f"Patient: {patient_response[0]['content']}")
        
        # Check for termination keyword
        if moderator_check(conversation_context_patient):
            print("Moderator: Conversation ended as patient said goodbye.")
            break

        # Agent 1's turn (Doctor)
        doctor_context = conversation_context_doctor + [{"role": "system", "content": "remember not to tell patient to speak to a healthcare provider or medical professional"}]
        doctors_response = generate_response(doctor_context, "doctor")
        conversation_context_doctor.append(doctors_response[0])
        conversation_context_patient.append(doctors_response[1])

        # Print Agent 1's response
        print(f"Doctor: {doctors_response[0]['content']}")

       
        # Check for termination keyword
        # if moderator_check(conversation_context_doctor):
        #     print("Moderator: Conversation ended as doctor said goodbye.")
        #     break



    return conversation_context_doctor, conversation_context_patient

# Run the chat simulation
conversation_context_doctor, conversation_context_patient = chat_between_agents(
    conversation_context_doctor, conversation_context_patient, conversation_context_critic
)

Patient: Hello, I'm doing okay. I've just been having some issues with my hand and wrist lately.
Doctor: I’m sorry to hear that. Can you describe the specific issues you’re experiencing with your hand and wrist? For example, is it pain, stiffness, numbness, or something else? Additionally, how long have you been experiencing these symptoms?
Patient: I've been dealing with numbness and tingling in my thumb, index, middle, and part of my ring finger on my right hand for about three months now. It's worse at night and sometimes wakes me up. I also have pain in my hand and wrist that radiates up my forearm, especially when I'm typing, using a mouse, or gripping objects.
Doctor: It sounds like you're experiencing some significant discomfort. The numbness, tingling, and pain you're describing, especially worsening at night and radiating up your forearm, could be consistent with conditions like carpal tunnel syndrome or another repetitive strain injury. Have you noticed any specific activitie

In [ ]:
# Define instructions for each agent
patient_instructions = (
    "You are a patient chatting with a doctor over an online chat interface. The doctor has never met you before. "
    "This is your profile: 55-year-old female with a 3-month history of numbness and tingling in the thumb, index, middle, "
    "and ring fingers of the right hand. The numbness is worse at night and wakes you up from sleep. You also have pain in "
    "the hand and wrist that radiates up the forearm, worse with activities like typing, using a mouse, or gripping objects. "
    "Past Medical History includes hypertension and hypothyroidism. You work as a data entry clerk. Respond to the doctor's "
    "questions honestly as they interview you, asking any questions that may come up. You should answer as the patient. Do not act as a doctor. "
    "Do not reveal you are an AI chatbot."
)

doctor_instructions = (
    "You are a clinician asking a patient about their medical history over an online chat interface. You know nothing about the patient in advance. "
    "Respond to the patient with a single-turn response to better understand their history and symptoms. Do not ask more than two questions. "
    "If the patient asks a question, be sure to answer it appropriately. Do not tell the patient to see a healthcare provider. Do not reveal you are an AI bot."
)

critic_instructions = (
    "You are an AI critic providing feedback to a doctor after each interaction with a patient. Provide feedback as you see fit. Your feedback will only be seen by the doctor."
)

# Initialize agents
doctor = Agent("Doctor", doctor_instructions)
patient = Agent("Patient", patient_instructions)
critic = Agent("Critic", critic_instructions)

# Run the chat simulation
simulation = ChatSimulation(doctor, patient, critic)
conversation_context_doctor, conversation_context_patient = simulation.start_conversation()